In [1]:
using Pkg
using IJulia
using DifferentialEquations
using Plots, CSV, XLSX, DataFrames
using Peaks, Statistics,Roots, NaNStatistics, Random
using StaticArrays
using IterTools
using Printf
using XLSX
using DataFrames

function freeR(rt,at,ks)
    return (rt .- at .- ks .+ sqrt.((at .- rt .- ks).^2 .+ 4*at*ks))/2
end
function freeA(rt,at,ks)
    return (at .- rt .- ks .+ sqrt.((at .- rt .- ks).^2 .+ 4*at*ks))/2
end
function compRA(rt,at,ks)
    return (at .+ rt .+ ks .- sqrt.((at .- rt .- ks).^2 .+ 4*at*ks))/2
end
function coeffI(rt,at,ka,ks,kb,kd)
    return (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)) ./ (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)*(ks*kd/(ka*kb)))
end
function coeffJ(rt,at,ka,ks,kb,kd)
    return (ks .+ ka .+ freeR(rt,at,ks)) ./ (ks .+ (ks*kd/kb) .+ freeR(rt,at,ks)*(ks*kd/(ka*kb)))
end
function tranBSD(rt,at,ka,ks,kb,kd)
    return (coeffI(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) ./ (1 .+ (coeffI(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) .+ (coeffJ(rt,at,ka,ks,kb,kd) .* freeA(rt,at,ks) ./ ka) .* (freeR(rt,at,ks) ./ kb))
end

function rhythm(t,amp,p,ph,lev)
    rate = (amp/2)*cos(2*pi*(t-ph)/p)+lev
    return rate
end
function modelDro!(dp,p,pa,t)
    (a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd) = pa
    dp[1] = tranBSD(p[3],AT,Ka,Ks,Kb,Kd)-b1*p[1]
    dp[2] = p[1]-b2*p[2]-a3*p[2]
    dp[3] = a3*p[2]-b3*p[3]
end
function modelWT!(dp,p,pa,t)
    (a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd,ampD,phD,ampT,phT,levD,levT) = pa                             # AT 변경해야 함. 
    dp[1] = tranBSD(p[3],AT,Ka,Ks,Kb,Kd)-rhythm(t,ampD,24,phD,levD)*p[1]                       # Decay
    dp[2] = rhythm(t,ampT,24,phT,levT)*p[1]-b2*p[2]-a3*p[2]                                   # Translation
    dp[3] = a3*p[2]-b3*p[3]
end
function cv(x)
    return std(x) / mean(x)
end

# findmaxima, findminima 리스트에서 주변 window 크기를 고려해서 극댓값, 극솟값 필터링하는 함수 
function find_significant_extrema(data, window_size=100)
    # Find all peaks and troughs
    peaks, _ = findmaxima(data)
    troughs, _ = findminima(data)
    # Filter significant peaks and troughs based on the surrounding window
    significant_peaks = filter(p -> is_significant_extremum(data, p, window_size, >), peaks)
    significant_troughs = filter(t -> is_significant_extremum(data, t, window_size, <), troughs)
    return significant_peaks, significant_troughs
end

# findmaxima, findminima 리스트에서 주변 window 크기를 고려해서 극댓값/극솟값 판별하는 함수 
function is_significant_extremum(data, index, window_size, compare_func)
    start_idx = max(1, index - window_size)
    end_idx = min(length(data), index + window_size)
    # Check if the current extremum is the most extreme value in the window --> 가장 극댓/극솟값 선택 
    return all(i -> i == index || compare_func(data[index], data[i]), start_idx:end_idx)
end

# peak_values, trough_values, peak_times, trough_times, amplitudes, periods 계산하는 함수
function analyze_oscillations(time, data, n_cycles=10)
    # Use find_significant_extrema to get only significant peaks and troughs
    significant_peaks, significant_troughs = find_significant_extrema(data)
    # Combine and sort all significant extrema chronologically
    extrema = sort(vcat(significant_peaks, significant_troughs))
    # Identify whether each extremum is a peak or trough
    is_peak = [i in significant_peaks for i in extrema]
    # Initialize arrays
    peak_values = Float64[]
    trough_values = Float64[]
    peak_times = Float64[]
    trough_times = Float64[]
    amplitudes = Float64[]
    periods = Float64[]
    # Track the last peak for period calculation
    last_peak_time = nothing
    # Analyze cycles
    cycle_count = 0
    for (i, idx) in enumerate(extrema)
        if cycle_count >= n_cycles
            break
        end
        if is_peak[i]
            push!(peak_values, data[idx])
            push!(peak_times, time[idx]) 
            if !isempty(trough_values)
                # Calculate amplitude
                push!(amplitudes, data[idx] - trough_values[end])
            end          
            if !isnothing(last_peak_time)
                # Calculate period
                push!(periods, time[idx] - last_peak_time)
                cycle_count += 1
            end
            last_peak_time = time[idx]
        else
            push!(trough_values, data[idx])
            push!(trough_times, time[idx])
        end
    end
    return peak_values, trough_values, peak_times, trough_times, amplitudes, periods
end
function calculate_pcm_SA_cost_v3(paInit)  

    a3, b1, b2, b3, AT, Ka, Ks, Kb, Kd = paInit

    time_range = 0:0.01:300
    bound = 300
    timept = collect(0:4:bound)
    
    prob = ODEProblem(modelDro!, [0,0,0], (0.0, 1200.0), paInit)
    detDro = solve(prob, saveat=0.01, Rosenbrock23())
    dp1_solution_pcm = [detDro[i][1] for i in 90001:(90001+bound*100)]
    dp3_solution_pcm = [detDro[i][3] for i in 90001:(90001+bound*100)]
    
    #--------------------pcm scale 계산해서 maximum을 1로 맞추는 부분-----------------------
    dp1_cycle_num = length(find_significant_extrema(dp1_solution_pcm)[1])
    dp3_cycle_num = length(find_significant_extrema(dp3_solution_pcm)[1])
    peaks_dp1, troughs_dp1, peak_times_dp1, trough_times_dp1, amplitudes_dp1, periods_dp1 = 
        analyze_oscillations(time_range, dp1_solution_pcm, dp1_cycle_num)
    peaks_dp3, troughs_dp3, peak_times_dp3, trough_times_dp3, amplitudes_dp3, periods_dp3 = 
        analyze_oscillations(time_range, dp3_solution_pcm, dp3_cycle_num)
    totalSACost = 10 
    if isempty(periods_dp1) || isempty(periods_dp3) || isempty(peak_times_dp1) || isempty(trough_times_dp1) || isempty(peak_times_dp3) || isempty(trough_times_dp3) 
        return totalSACost
    end
    pcmMperScale = maximum(peaks_dp1)
    pcmPERScale = maximum(peaks_dp3)
    dp1_solution_pcm = dp1_solution_pcm./maximum(peaks_dp1)
    dp3_solution_pcm = dp3_solution_pcm./maximum(peaks_dp3)
    # --------------------------------------------------------------------------------------

    # -----------------리듬 주기, amplitude CV, period CV 계산하는 부분  --------------------
    dp1_cycle_num = length(find_significant_extrema(dp1_solution_pcm)[1])
    dp3_cycle_num = length(find_significant_extrema(dp3_solution_pcm)[1])
    peaks_dp1, troughs_dp1, peak_times_dp1, trough_times_dp1, amplitudes_dp1, periods_dp1 = 
        analyze_oscillations(time_range, dp1_solution_pcm, dp1_cycle_num)
    peaks_dp3, troughs_dp3, peak_times_dp3, trough_times_dp3, amplitudes_dp3, periods_dp3 = 
        analyze_oscillations(time_range, dp3_solution_pcm, dp3_cycle_num)
    len_dp1 = min(length(amplitudes_dp1), length(peaks_dp1))
    len_dp3 = min(length(amplitudes_dp3), length(peaks_dp3))
    relamp_1 = amplitudes_dp1[1:len_dp1] ./ peaks_dp1[1:len_dp1]
    relamp_3 = amplitudes_dp3[1:len_dp3] ./ peaks_dp3[1:len_dp3]
    if isempty(periods_dp1) || isempty(periods_dp3) || isempty(relamp_1) || isempty(relamp_3) || isempty(peak_times_dp1) || isempty(trough_times_dp1) || isempty(peak_times_dp3) || isempty(trough_times_dp3) 
        return totalSACost
    end
    cost_periods_pcmMper = sqrt(sum((1 .- periods_dp1 ./ 24).^2))    
    cost_periods_pcmPER = sqrt(sum((1 .- periods_dp3 ./ 24).^2))
    if cost_periods_pcmPER > 0.1 || cost_periods_pcmMper > 0.1 
        return totalSACost
    end
    cv_relamp_1 = cv(relamp_1)
    cv_relamp_3 = cv(relamp_3)
    cv_periods_pcmMper = cv(periods_dp1)
    cv_periods_pcmPER = cv(periods_dp3)
    # --------------------------------------------------------------------------------------

    pcmMper = [0.9417318265372026, 1.000, 0.6863256043100736, 0.6500461271162583, 0.6613263978206749, 0.8107829120597052]
    pcmPER = [0.775786555953261, 0.675942056075872, 1.0000, 0.8581647488931649, 0.5882726262405783, 0.5400946014398237]
    wtMper = [0.6010842804859873, 0.44965606490502713, 0.4500873206666041, 0.8491425945450632, 0.9595440695087687, 1.108686629553371]
    wtPER = [0.5184852832893596, 0.36710148731046305, 0.26384484454227036, 0.2173993457282554, 0.2027945985939066, 0.22124481729244966]

    if cv_relamp_1 > 0.1 || cv_relamp_3 > 0.1 || cost_periods_pcmPER > 0.1 || cost_periods_pcmMper > 0.1 || cv_periods_pcmMper > 0.1 || cv_periods_pcmPER > 0.1
        return totalSACost
    end

    # RMS 계산 전에 peak점을 맞추도록, 기존 pcm Mper, pcm PER의 peak를 계산하는 부분  
    filtered_peaks_pcm_mRNA = [x for x in peak_times_dp1 if 260 <= x <= 288]
    filtered_troughs_pcm_mRNA = [x for x in trough_times_dp1 if 260 <= x <= 288]
    filtered_peaks_pcm_PER = [x for x in peak_times_dp3 if 260 <= x <= 288]
    filtered_troughs_pcm_PER = [x for x in trough_times_dp3 if 260 <= x <= 288]
    if isempty(filtered_peaks_pcm_mRNA) || isempty(filtered_troughs_pcm_mRNA) || isempty(filtered_peaks_pcm_PER) || isempty(filtered_troughs_pcm_PER) 
        return totalSACost
    end
    pcmMrnaMaxT = minimum(filtered_peaks_pcm_mRNA)
    pcmMrnaMinT = maximum(filtered_troughs_pcm_mRNA)
    pcmPERMaxT = minimum(filtered_peaks_pcm_PER)
    pcmPERMinT = maximum(filtered_troughs_pcm_PER)

    # 기존의 peak와 데이터의 peak를 서로 비교해서, peak점이 겹치도록 pcm Mper, pcm PER를 shift하는 부분
    dp1_shift = 0
    if (268 - pcmMrnaMaxT) > 0  
        dp1_shift = Int(trunc(-(268 - pcmMrnaMaxT) * 100))
    else                        
        dp1_shift = -Int(trunc(2400 + (268 - pcmMrnaMaxT) * 100))
    end
    dp2_shift = 0
    if (272 - pcmPERMaxT) > 0  
        dp2_shift = Int(trunc(-(272 - pcmPERMaxT) * 100))
    else                        
        dp2_shift = -Int(trunc(2400 + (272 - pcmPERMaxT) * 100))
    end
    dp1_solution_pcm_shifted = detDro[1, 90001+dp1_shift:(90001+bound*100+dp1_shift)]/pcmMperScale
    dp3_solution_pcm_shifted = detDro[3, 90001+dp2_shift:(90001+bound*100+dp2_shift)]/pcmPERScale

    # peak점을 겹치도록 shift한 pcm Mper, pcm PER에서 각각 시간값 [264, 268, 272, 276, 280, 284]에 대응하는 y값들 계산하는 부분
    detDro1_shifted_timepoint_vals = []  
    detDro3_shifted_timepoint_vals = []  
    indices = [264, 268, 272, 276, 280, 284] .*= 100
    for idx in indices
        push!(detDro1_shifted_timepoint_vals, dp1_solution_pcm_shifted[idx])  
        push!(detDro3_shifted_timepoint_vals, dp3_solution_pcm_shifted[idx])
    end

    # 끝부분 시간대 [264, 288] 에서 mRNA 최소점 시간, mRNA 최대점 시간, PER 최소점 시간, PER 최대점 시간 구하는 부분.
    peaks_dp1_shifted, troughs_dp1_shifted, peak_times_dp1_shifted, trough_times_dp1_shifted, amplitudes_dp1_shifted, periods_dp1_shifted = 
        analyze_oscillations(time_range, dp1_solution_pcm_shifted, dp1_cycle_num)
    peaks_dp3_shifted, troughs_dp3_shifted, peak_times_dp3_shifted, trough_times_dp3_shifted, amplitudes_dp3_shifted, periods_dp3_shifted = 
        analyze_oscillations(time_range, dp3_solution_pcm_shifted, dp3_cycle_num) 
    filtered_peaks_pcm_mRNA_shifted = [x for x in peak_times_dp1_shifted if 264 <= x <= 288]
    filtered_troughs_pcm_mRNA_shifted = [x for x in trough_times_dp1_shifted if 264 <= x <= 288]
    filtered_peaks_pcm_PER_shifted = [x for x in peak_times_dp3_shifted if 264 <= x <= 288]
    filtered_troughs_pcm_PER_shifted = [x for x in trough_times_dp3_shifted if 264 <= x <= 288]
    totalSACost = 10
    if isempty(filtered_peaks_pcm_mRNA_shifted) || isempty(filtered_troughs_pcm_mRNA_shifted) || isempty(filtered_peaks_pcm_PER_shifted) || isempty(filtered_troughs_pcm_PER_shifted)
        return totalSACost
    end
    pcmMrnaMaxT_shifted = minimum(filtered_peaks_pcm_mRNA_shifted)
    pcmMrnaMinT_shifted = minimum(filtered_troughs_pcm_mRNA_shifted)
    pcmPERMaxT_shifted = minimum(filtered_peaks_pcm_PER_shifted)
    pcmPERMinT_shifted = minimum(filtered_troughs_pcm_PER_shifted)

    # ---------------> Cost 1번, pcm mRNA RMS ---------------
    costPcmMrna_RMS = 0
    for i in 1:6
        costPcmMrna_RMS += 0.05*(1 - detDro1_shifted_timepoint_vals[i] / pcmMper[i])^2
    end
    # ---------------> Cost 2번, pcm PER RMS ---------------
    costPcmPER_RMS = 0
    for i in 1:6
        costPcmPER_RMS += 0.05*(1 - detDro3_shifted_timepoint_vals[i] / pcmPER[i])^2
    end
    # ---------> Cost 3번,  costPcmMrna_minT_maxT (mPER에서 minT-maxT를 맞추는 cost)---------------
    costPcmMrna_minT_maxT = 0.0    
    costPcmMrna_minT_maxT += (1 - (abs(pcmMrnaMinT_shifted - pcmMrnaMaxT_shifted)) / 8)^2  # minT - maxT 시간 차이 cost

    # --------> Cost 4번, costPcmPER_minT_maxT (PER에서 minT-maxT를 맞추는 cost)----------
    costPcmPER_minT_maxT = 0.0     
    costPcmPER_minT_maxT += (1 - (abs(pcmPERMinT_shifted - pcmPERMaxT_shifted)) / 12)^2    # minT - maxT cost 더하는 부분 

    # --------> Cost 5번, cost_minimum_difference (minimum을 맞추는 cost)---------------        
    cost_minimum_difference = ((1-minimum(dp1_solution_pcm_shifted)/0.6500461271162583)^2 + (1-minimum(dp3_solution_pcm_shifted)/0.5400946014398237)^2)    

    # ----------------> Total Cost ----------------------------------
    totalSACost = sqrt(costPcmMrna_RMS + costPcmPER_RMS + costPcmMrna_minT_maxT + costPcmPER_minT_maxT + cost_minimum_difference)
        
    return totalSACost
end

function perturb(pa)
    newpa = [pa[i] * exp(0.1 * rand(1)[1] - 0.05) for i in 1:length(pa)]
    return newpa
end
function simulated_annealing(objective_function, initial_state, max_iterations, initial_temp, alpha)
    # Initialize the state
    current_state = initial_state
    current_value = objective_function(current_state)
    best_state = current_state
    best_value = current_value
    
    # Initialize temperature
    temp = initial_temp
    
    for i in 1:max_iterations
        # Generate a new candidate state
        candidate_state = perturb(current_state)
        candidate_value = objective_function(candidate_state)
        
        # Calculate the change in the objective function value
        delta_value = candidate_value - current_value
        
        # Decide whether to accept the new candidate state
        if delta_value < 0 || exp(-delta_value / temp) > rand()
            current_state = candidate_state
            current_value = candidate_value
        end
        
        # Update the best state found so far
        if current_value < best_value
            best_state = current_state
            best_value = current_value
        end
        
        # Decrease the temperature
        temp *= alpha
        
        # Print progress
        if i%100==0
            println("Iteration $i: Para = $candidate_state, Candidate = $candidate_value, Current = $current_value, Best value = $best_value, Temperature = $temp")
        end
    end
    
    return best_state, best_value
end




simulated_annealing (generic function with 1 method)

In [5]:

pcmMper = [0.9417318265372026, 1.000, 0.6863256043100736, 0.6500461271162583, 0.6613263978206749, 0.8107829120597052]
pcmPER = [0.775786555953261, 0.675942056075872, 1.0000, 0.8581647488931649, 0.5882726262405783, 0.5400946014398237]
wtMper = [0.6010842804859873, 0.44965606490502713, 0.4500873206666041, 0.8491425945450632, 0.9595440695087687, 1.108686629553371]
wtPER = [0.5184852832893596, 0.36710148731046305, 0.26384484454227036, 0.2173993457282554, 0.2027945985939066, 0.22124481729244966]

# Parameters order: a3,b1,b2,b3,AT,Ka,Ks,Kb,Kd
# pcmPaInit = [1.259161147, 0.022480368, 0.121116305, 0.941907467, 0.094151061, 5.19e-5, 3.01e-9, 7.67e-5, 20.62876348]
pcmPaInit = [0.03506779994821416, 0.03534939638143893, 0.23124081209936562, 0.3145194084346986, 2.0550042876829218, 0.0005169795651216473, 5.368831175403542e-10, 3.960562426536927e-5, 24.849093585591426]


best_state, best_value = simulated_annealing(calculate_pcm_SA_cost_v3, pcmPaInit, 100000, 1, 0.9995)

best_state, best_value


Iteration 100: Para = [0.03772782496447821, 0.04675185891723221, 0.1808843484419439, 0.26121583515612623, 2.923056846470739, 0.0004369241661970481, 5.081203868746458e-10, 5.285088019800097e-5, 21.136523625587685], Candidate = 10, Current = 1.340841282761287, Best value = 1.1855688565160103, Temperature = 0.951217530242334
Iteration 200: Para = [0.04254526666302651, 0.04006157896832565, 0.19357790121931495, 0.270309508316381, 4.099927751729763, 0.00033443399058610086, 3.094676473938775e-10, 6.556792190235136e-5, 21.15433050773459], Candidate = 1.3102996709432746, Current = 1.3102996709432746, Best value = 1.1855688565160103, Temperature = 0.9048147898403269
Iteration 300: Para = [0.0537236391997994, 0.041574366983486676, 0.2740108590289756, 0.18591795533891425, 4.297478460240645, 0.00027791826351355006, 3.2740092043797744e-10, 7.739334303114149e-5, 16.32578364858363], Candidate = 10, Current = 1.442846424835595, Best value = 1.0835386251437265, Temperature = 0.8606756897186528
Iteration

([0.17815862684250944, 0.07590905720083663, 0.038700871702774316, 0.20424307667459626, 32.58125182818706, 0.0006418303096383012, 5.307206057832601e-9, 0.0011003978008460717, 0.10999921527187113], 0.5853090107400812)